### DB 연결

In [17]:
import pandas as pd

file_path="dataset/diabetes.csv"
df= pd.read_csv(file_path)

In [18]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
import pymysql

In [20]:
def connect_to_mariadb():
    return pymysql.connect(
        host='localhost',
        user='root',
        password='00000000',
        database='test',
        charset='utf8mb4'
    )

In [21]:
connection = connect_to_mariadb()
print("MariaDB에 연결되었습니다!")
connection.close()

MariaDB에 연결되었습니다!


In [22]:
def create_table_from_csv(connection, df, table_name):
    cursor = connection.cursor()
    
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        if "int" in str(dtype):
            col_type = "INT"
        elif "float" in str(dtype):
            col_type = "FLOAT"
        else:
            col_type = "VARCHAR(255)"  # 문자열 타입 처리
        columns.append(f"`{col}` {col_type}")

    columns_sql = ", ".join(columns)
    create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_sql});"

    # 테이블 생성 실행
    cursor.execute(create_table_sql)
    connection.commit()
    print(f"테이블 '{table_name}'이(가) 생성되었습니다.")

In [23]:
table_name = "diabetes"  # 생성할 테이블 이름
connection = connect_to_mariadb()
create_table_from_csv(connection, df, table_name)
connection.close()

테이블 'diabetes'이(가) 생성되었습니다.


In [24]:
def insert_data_to_table(connection, df, table_name):
    cursor = connection.cursor()

    # 데이터 삽입 SQL 동적 생성
    placeholders = ", ".join(["%s"] * len(df.columns))
    insert_sql = f"INSERT INTO `{table_name}` ({', '.join(df.columns)}) VALUES ({placeholders})"

    # 데이터 삽입 실행
    for _, row in df.iterrows():
        cursor.execute(insert_sql, tuple(row))

    connection.commit()
    print(f"테이블 '{table_name}'에 데이터가 삽입되었습니다.")

In [25]:
connection = connect_to_mariadb()
insert_data_to_table(connection, df, table_name)
connection.close()

테이블 'diabetes'에 데이터가 삽입되었습니다.


In [26]:
def fetch_data_from_table(connection, table_name):
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, connection)
    return df

In [27]:
connection = connect_to_mariadb()
table_name = "diabetes"  # 불러올 테이블 이름

In [28]:
df_from_db = fetch_data_from_table(connection, table_name)

C:\Users\human-14\AppData\Local\Temp\ipykernel_5620\2398583656.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [29]:
print("MariaDB에서 불러온 데이터:")
print(df_from_db.head())

MariaDB에서 불러온 데이터:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [30]:
connection.close()